<a href="https://colab.research.google.com/github/22mja01/macc-skip-tracing/blob/main/scrapeak_skip_tracing_people_search_scraper.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/drive/19qm5MPx4UZqrUz97TRjCzL5HNm0u-m8k?usp=sharing)
# Scrapeak | Skip Tracing - People Search Scraper

## Overview
| Detail Tag            | Information                                                                                        |
|-----------------------|----------------------------------------------------------------------------------------------------|
| Originally Created By | Ariel Herrera arielherrera@analyticsariel.com |
| External References   | API |
| Input Datasets        | Source name |
| Output Datasets       | Source name |
| Input Data Source     | Pandas DataFrame |
| Output Data Source    | Pandas DataFrame |

## History
| Date         | Developed By  | Reason                                                |
|--------------|---------------|-------------------------------------------------------|
| 1st Sep 2023 | Ariel Herrera | Create notebook. |

## Getting Started
1. Copy this notebook -> File -> Save a Copy in Drive
2. Directions

## Useful Resources
- [Google Colab Cheat Sheet](https://towardsdatascience.com/cheat-sheet-for-google-colab-63853778c093)
- [Scrapeak Documentation](https://docs.scrapeak.com/people-search-scraper/overview)

## <font color="blue">Install Packages</font>

In [ ]:
!pip install fuzzywuzzy -q
!pip install python-Levenshtein -q

## <font color="blue">Imports</font>

In [ ]:
import requests
import pandas as pd
from getpass import getpass
from google.colab import files
from fuzzywuzzy import fuzz
from fuzzywuzzy import process

## <font color="blue">Locals & Constants</font>

In [ ]:
api_key = getpass('Enter the scrapeak api key: ')

Enter the scrapeak api key: ··········


## <font color="blue">Data</font>

### Case Study: Pre-Foreclosures
Gather contact information (skip-trace) a list of property owners whose properties are in pre-foreclosure.
<br>
<b>Location:</b> Cleveland, Ohio
<br>
<b>Resource:</b> [Pre-foreclosure schedule](https://cpdocket.cp.cuyahogacounty.us/sheriffsearch/search.aspx)
<br>
<br>
<b>Foreclosure Background</b>

Pre-foreclosure is the period of time before a home is foreclosed on. It begins when a homeowner misses a mortgage payment and the lender sends a notice of default. The homeowner then has a certain amount of time to bring the loan current, usually 30-90 days. If the homeowner does not bring the loan current, the lender can begin the foreclosure process.

There are a few things that can happen during pre-foreclosure. The homeowner may be able to work with the lender to modify the loan terms or sell the property. The lender may also choose to sell the property through a short sale, which is when the property is sold for less than the amount owed on the loan.

If the homeowner does not take any action, the lender will eventually foreclose on the property. This means that the lender will take ownership of the property and sell it at auction. The homeowner will still be responsible for paying the remaining balance on the loan, even if they no longer own the property.


In [ ]:
# sample dataset of pre-foreclosures
pre_foreclosures_dict = {
    '0': {
        'address': '4110 WEST 143RD STREET',
        'location': 'CLEVELAND, OH 44135',
        'appraised_value': 145000,
        'opening_bid': 96667
    },
    '1': {
        'address': '6961 MAPLEWOOD ROAD',
        'location': 'PARMA HEIGHTS, OH 44130',
        'appraised_value': 190000,
        'opening_bid': 126667
    },
    '2': {
        'address': '14315 DARWIN AVENUE',
        'location': 'CLEVELAND, OH 44110',
        'appraised_value': 50000,
        'opening_bid': 33334
    },
    '3': {
        'address': '13577 FOXCROFT DRIVE',
        'location': 'GARFIELD HEIGHTS, OH 44125',
        'appraised_value': 155000,
        'opening_bid': 103334
    }
}

# transform dictionary into a dataframe
df_pf = pd.DataFrame(pre_foreclosures_dict).T
df_pf

,address,location,appraised_value,opening_bid
0,4110 WEST 143RD STREET,"CLEVELAND, OH 44135",145000,96667
1,6961 MAPLEWOOD ROAD,"PARMA HEIGHTS, OH 44130",190000,126667
2,14315 DARWIN AVENUE,"CLEVELAND, OH 44110",50000,33334
3,13577 FOXCROFT DRIVE,"GARFIELD HEIGHTS, OH 44125",155000,103334


### 1) Search by Addresss 🏠
[Api Docs](https://docs.scrapeak.com/people-search-scraper/endpoints/searchbyaddress) | Search for owner information by address

In [ ]:
address = "4110 WEST 143RD STREET"
location = "CLEVELAND, OH 44135"

# API endpoint
api_url = "https://app.scrapeak.com/v1/scrapers/people_search/search"

parameters = {
    "api_key": api_key,
    "search_by": "address",
    "address": address,
    "location": location
}

# Make the API request
response = requests.get(api_url, params=parameters)
response_content = response.text

print(response_content)

{"is_success":true,"data":[{"fullName":"Eladio Pena","location":"Medina, OH","coordinate_lat":"41.142953","coordinate_lng":"-81.894594","age":"86","currentHomeAddress":"211 Abbeyville Rd, Medina OH 44256","pastAddresses":["4110 W 143rd St, Cleveland OH 44135","7322 Wakefield Ave, Cleveland OH 44102","31281 Hc 1, Juana Diaz PR 00795","14219 Triskett Rd, Unit 303C, Cleveland OH 44111","41 Calle 4, Juana Diaz PR 00795","1372 W 80th St, Unit 107, Cleveland OH 44102"],"phone":["(216) 241-2424","(216) 326-3987","(216) 334-1829","(216) 235-2835"],"aka":["Eladio Penajr • Eladio Pena • Eladio P Pena"],"relatives":["Celia Pena • Eladia Pena • Eladio Pena • Elizabeth Pena • Elizabeth Pena • Matiwilda Hanbaum • Mercedes Pena • Mohannad Moualem • Ramona Pena • Adalberto Rivera • Aleya Moualem • Amal Moualem • Angelina Rivera • Carmen Cruz"]},{"fullName":"Iutka Fenyes","location":"Cleveland, OH","coordinate_lat":"41.142953","coordinate_lng":"-81.894594","age":"64","currentHomeAddress":"4110 W 143rd 

In [ ]:
# transform into a json obj
response.json()

{'is_success': True,
 'data': [{'fullName': 'Eladio Pena',
   'location': 'Medina, OH',
   'coordinate_lat': '41.142953',
   'coordinate_lng': '-81.894594',
   'age': '86',
   'currentHomeAddress': '211 Abbeyville Rd, Medina OH 44256',
   'pastAddresses': ['4110 W 143rd St, Cleveland OH 44135',
    '7322 Wakefield Ave, Cleveland OH 44102',
    '31281 Hc 1, Juana Diaz PR 00795',
    '14219 Triskett Rd, Unit 303C, Cleveland OH 44111',
    '41 Calle 4, Juana Diaz PR 00795',
    '1372 W 80th St, Unit 107, Cleveland OH 44102'],
   'phone': ['(216) 241-2424',
    '(216) 326-3987',
    '(216) 334-1829',
    '(216) 235-2835'],
   'aka': ['Eladio Penajr • Eladio Pena • Eladio P Pena'],
   'relatives': ['Celia Pena • Eladia Pena • Eladio Pena • Elizabeth Pena • Elizabeth Pena • Matiwilda Hanbaum • Mercedes Pena • Mohannad Moualem • Ramona Pena • Adalberto Rivera • Aleya Moualem • Amal Moualem • Angelina Rivera • Carmen Cruz']},
  {'fullName': 'Iutka Fenyes',
   'location': 'Cleveland, OH',
   'c

In [ ]:
# transform into a json obj
df_skip_trace_all = pd.json_normalize(response.json()['data'])
print(f'Num of records: {len(df_skip_trace_all)}')
print(f'Num of columns: {len(df_skip_trace_all.columns)}')
df_skip_trace_all

Num of records: 10
Num of columns: 10


,fullName,location,coordinate_lat,coordinate_lng,age,currentHomeAddress,pastAddresses,phone,aka,relatives
0,Eladio Pena,"Medina, OH",41.142953,-81.894594,86,"211 Abbeyville Rd, Medina OH 44256","[4110 W 143rd St, Cleveland OH 44135, 7322 Wak...","[(216) 241-2424, (216) 326-3987, (216) 334-182...",[Eladio Penajr • Eladio Pena • Eladio P Pena],[Celia Pena • Eladia Pena • Eladio Pena • Eliz...
1,Iutka Fenyes,"Cleveland, OH",41.142953,-81.894594,64,"4110 W 143rd St, Cleveland OH 44135","[5019 Royalton Rd, North Royalton OH 44133, 58...","[(440) 628-9010, (216) 671-3525, (330) 239-043...",[Jutka M Fenyes • Iutka Fenyes • Jutka Fenyes ...,[Alexandru Fenyes • Judith Fenyes • Sandor Fen...
2,Gabor Dioszegi,"Strongsville, OH",41.142953,-81.894594,50,"20941 Drake Rd, Strongsville OH 44149","[4110 W 143rd St, Cleveland OH 44135, 14738 E ...","[(440) 773-8055, (440) 773-8805, (216) 509-588...",[Garbor Dioszegi],[Eszter Dioszegineekes]
3,V Fodor Erika,"Cleveland, OH",41.142953,-81.894594,34,"4110 W 143rd St, Cleveland OH 44135","[25 Gish Rd, Rittman OH 44270]",NaN,NaN,NaN
4,Bela Czirjak,"Columbia Station, OH",41.142953,-81.894594,49,"9933 Station Rd, Columbia Station OH 44028","[4110 W 143rd St, Cleveland OH 44135, 4525 W 1...",[(216) 598-0173],[Bela Czirjak • Bela C Czirjack • Bela J Czirjak],[Bela Czirjak • Ilona Czirjak • Tamas Czirjak ...
5,Bela Czirjak,"Cleveland, OH",41.142953,-81.894594,75,"11800 Brookpark Rd, Unit 126, Cleveland OH 44130","[4525 W 146th St, Cleveland OH 44135, 738 Toll...","[(440) 237-9436, (440) 590-5159, (216) 476-000...",[Bela Czirjak • Bela L Czirjak • Bela Cziriak],[Ilona Czirjak • Bela Czirjak • Tamas Czirjak ...
6,Eszter Dioszegineekes,"Strongsville, OH",41.142953,-81.894594,43,"20941 Drake Rd, Strongsville OH 44149","[4110 W 143rd St, Cleveland OH 44135, 14738 E ...","[(440) 708-4256, (440) 625-0950, (216) 862-5574]",[Eszter D Ekes • Eszter Dioszegi • Eszter Ekes...,[Gabor Dioszegi]
7,Priscilla Wieser,"Rotonda West, FL",41.142953,-81.894594,73,"209 W Pine Valley Ln, Rotonda West FL 33947","[7305 Skycrest St, Englewood FL 34224, 4110 W ...","[(440) 236-8706, (440) 236-8535, (732) 441-494...",[Priscilla T Dezso • Priscilla Weiser • Prisci...,[Andrea Dezso • Anna Wieser • Anna Wieser • Bo...
8,Patricia Caddick,"Euclid, OH",41.142953,-81.894594,Deceased (1960 - 2004),"431 Clearview Dr, Unit C, Euclid OH 44123","[4110 W 143rd St, Cleveland OH 44135, 17704 Ar...","[(216) 289-1841, (216) 671-1857, (216) 671-750...",[Patircia E Caddick • Pat Caddick],[Angelica Caddick • Scott Caddick • Harry Cadd...
9,Jutka Fenyes,"Wadsworth, OH",41.142953,-81.894594,NaN,"5850 Beach Rd, Wadsworth OH 44281","[5019 Royalton Rd, North Royalton OH 44133, 41...","[(216) 671-3525, (330) 239-0436, (234) 248-4031]",[Judka Fenyes],[Alexandru Fenyes • Judith Fenyes • Jutka Feny...


<b>Fuzzy Wuzzy String Matching</b>
<br>
The fuzz ratio and fuzz partial ratio are two string similarity measures in Python. They are both implemented in the FuzzyWuzzy library.

The fuzz ratio measures the similarity between two strings by comparing the number of matching characters and the number of differing characters.

The fuzz partial ratio only considers the longest common substring between the two strings.

In [ ]:
print('Subj address:', address)
similar_address = "4110 W 143rd St, Cleveland OH 44135"
print('Similar address:', similar_address)
formatted_similar_address = similar_address.upper().split(',')[0]
print('  Formatted:', formatted_similar_address)
print('Fuzz ratio:', fuzz.ratio(address, formatted_similar_address))
print('Fuzz partial ratio:', fuzz.partial_ratio(address, formatted_similar_address))

Subj address: 4110 WEST 143RD STREET
Similar address: 4110 W 143rd St, Cleveland OH 44135
  Formatted: 4110 W 143RD ST
Fuzz ratio: 81
Fuzz partial ratio: 80


In [ ]:
# skip trace features
df_st_feat = df_skip_trace_all.copy()
df_st_feat['fuzz_ratio_score'] = df_st_feat.apply(lambda x: fuzz.ratio(address, x['currentHomeAddress'].upper().split(',')[0]), axis=1)
df_st_feat['fuzz_ratio_rank'] = df_st_feat['fuzz_ratio_score'].rank(ascending=False, method='min')
df_st_feat = df_st_feat.loc[
    (df_st_feat['fuzz_ratio_rank'] == 1) & # get top scoring string
    (~df_st_feat['phone'].isnull()) # filter where contact info (phone) exist
]
df_st_feat

,fullName,location,coordinate_lat,coordinate_lng,age,currentHomeAddress,pastAddresses,phone,aka,relatives,fuzz_ratio_score,fuzz_ratio_rank
1,Iutka Fenyes,"Cleveland, OH",41.142953,-81.894594,64,"4110 W 143rd St, Cleveland OH 44135","[5019 Royalton Rd, North Royalton OH 44133, 58...","[(440) 628-9010, (216) 671-3525, (330) 239-043...",[Jutka M Fenyes • Iutka Fenyes • Jutka Fenyes ...,[Alexandru Fenyes • Judith Fenyes • Sandor Fen...,81,1.0


<b>Skip Trace List of Addresses</b>

In [ ]:
def scrapeak_skip_trace_address(api_key, address, location):
  # API endpoint
  api_url = "https://app.scrapeak.com/v1/scrapers/people_search/search"

  parameters = {
      "api_key": api_key,
      "search_by": "address",
      "address": address,
      "location": location
  }

  # Make the API request
  response = requests.get(api_url, params=parameters)
  return response.json()


def transform_json_to_df(json_obj):
  # pandas dataframe
  df = pd.json_normalize(json_obj['data'])
  # features
  df['fuzz_ratio_score'] = df.apply(lambda x: fuzz.ratio(address, x['currentHomeAddress'].upper().split(',')[0]), axis=1)
  df['fuzz_ratio_rank'] = df['fuzz_ratio_score'].rank(ascending=False, method='min')
  return df.loc[
      (df['fuzz_ratio_rank'] == 1) & # get top scoring string
      (~df['phone'].isnull()) # filter where contact info (phone) exist
  ]

In [ ]:
df_pf

,address,location,appraised_value,opening_bid
0,4110 WEST 143RD STREET,"CLEVELAND, OH 44135",145000,96667
1,6961 MAPLEWOOD ROAD,"PARMA HEIGHTS, OH 44130",190000,126667
2,14315 DARWIN AVENUE,"CLEVELAND, OH 44110",50000,33334
3,13577 FOXCROFT DRIVE,"GARFIELD HEIGHTS, OH 44125",155000,103334


In [ ]:
# list of dataframes
df_list = []

# iterate through list of properties
for index, row in df_pf.iterrows():
    address = row['address']
    location = row['location']
    print(f'Getting data for {address}, {location}')
    # get json obj
    response_json = scrapeak_skip_trace_address(api_key, address, location)
    # transform into a dataframe
    _df = transform_json_to_df(response_json)
    # append to list
    df_list.append(_df)

Getting data for 4110 WEST 143RD STREET, CLEVELAND, OH 44135
Getting data for 6961 MAPLEWOOD ROAD, PARMA HEIGHTS, OH 44130
Getting data for 14315 DARWIN AVENUE, CLEVELAND, OH 44110
Getting data for 13577 FOXCROFT DRIVE, GARFIELD HEIGHTS, OH 44125


In [ ]:
# join list of dfs
df_skip_trace_addr = pd.concat(df_list)
df_skip_trace_addr

,fullName,location,coordinate_lat,coordinate_lng,age,currentHomeAddress,pastAddresses,phone,aka,relatives,fuzz_ratio_score,fuzz_ratio_rank
1,Iutka Fenyes,"Cleveland, OH",41.142953,-81.894594,64,"4110 W 143rd St, Cleveland OH 44135","[5019 Royalton Rd, North Royalton OH 44133, 58...","[(440) 628-9010, (216) 671-3525, (330) 239-043...",[Jutka M Fenyes • Iutka Fenyes • Jutka Fenyes ...,[Alexandru Fenyes • Judith Fenyes • Sandor Fen...,81,1.0
0,Daisy Valverde,"Parma Heights, OH",41.375178,-81.780966,42,"6961 Maplewood Rd, Parma Heights OH 44130","[6961 Maplewood Rd, Cleveland OH 44130, 8000 W...","[(440) 253-8068, (216) 937-5540, (216) 347-013...",[Daisy Roberts • Daisy Velez • Daisy N Valverd...,[Alejandro Velez • Daniel Roberts • David Vele...,94,1.0
3,Maria Wozniak,"Parma Heights, OH",41.375178,-81.780966,100,"6961 Maplewood Rd, Parma Heights OH 44130","[4160 W 56th St, Cleveland OH 44144, 17 West D...","[(440) 885-2158, (216) 771-1740, (216) 888-5153]",[A M Wozniak • Maria Nydza • Marie Wozniak • M...,[Alexander Nydza • Anna Nydza • Michael Nydza ...,94,1.0
8,Gerke Klemens,"Parma Heights, OH",41.375178,-81.780966,98,"6961 Maplewood Rd, Parma Heights OH 44130","[1473 Wagar Ave, Cleveland OH 44107, 6961 Mapl...","[(440) 885-2158, (216) 228-2316]",[Klemens Gerke],NaN,94,1.0
0,Janet Hawkins,"Cleveland, OH",41.565050,-81.582231,58,"14315 Darwin Ave, Cleveland OH 44110","[933 Hanley Rd, Cleveland OH 44124, 933 Hanley...","[(440) 395-3046, (216) 761-5532, (216) 225-809...",[Janet Hawkins • Janet Marie Hawkins • Janet M...,[Jewell Hawkins • Bobbie Alexander • Debra Duk...,91,1.0
2,Eddie Hawkins,"Cleveland, OH",41.565050,-81.582231,70,"14315 Darwin Ave, Cleveland OH 44110","[15419 Ridpath Ave, Cleveland OH 44110, 940 Id...",[(216) 531-3751],[Eddie J Hawkins • Eddie James Hawkins • Edwar...,[Eddie Hawkins • Janet Hawkins • Janet Hawkins...,91,1.0
3,Janet Hawkins,"Cleveland, OH",41.565050,-81.582231,58,"14315 Darwin Ave, Cleveland OH 44110",NaN,"[(216) 862-2632, (216) 761-5532]",[Janet M Hawkins • Janette Hawkins],[Eddie Hawkins • Janet Hawkins • Jeremy Hawkin...,91,1.0
0,Mario Little,"Garfield Heights, OH",41.395335,-81.589405,67,"13577 Foxcroft Dr, Garfield Heights OH 44125","[13577 Foxcroft Dr, Cleveland OH 44125, Po Box...","[(216) 789-5252, (216) 561-0119, (423) 265-509...",[Mario Elmer Little • Mario Little • Mario J L...,[Anthony Little • Erica Hicks • Johonna Little...,92,1.0
1,Rio Bell,"Garfield Heights, OH",41.395335,-81.589405,31,"13577 Foxcroft Dr, Garfield Heights OH 44125","[1225 Jackson Ln, Unit 130, Middletown OH 4504...","[(513) 217-1766, (513) 300-6805, (513) 217-093...",[Rio S Bell • Rio D Bell],[Arianah Bell • Edna Rogers • Alphonso Rogers ...,92,1.0
2,Leon Fowles,"Cleveland, OH",41.395335,-81.589405,Deceased (1917 - 2003),"13577 Foxcroft Dr, Cleveland OH 44125","[13577 Foxcroft Dr, Garfield Heights OH 44125,...",[(216) 662-0288],[Leon James Fowles • Leon Fowles],NaN,92,1.0


In [ ]:
# [OPTIONAL] export into a csv file
file_name = 'output.csv'
df_skip_trace_addr.to_csv(file_name, index=False)
files.download(file_name)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

### 2) Search by Name 👤
[Api Docs](https://docs.scrapeak.com/people-search-scraper/endpoints/searchbyname) | Search contact information by owner name

In [ ]:
name = "Mario Little"
location = "Garfield Heights OH 44125"

#/ API endpoint
api_url = "https://app.scrapeak.com/v1/scrapers/people_search/search"

parameters = {"api_key": api_key, "search_by":"name", "name": name, "location":location}

#/ Make the API request
response = requests.get(api_url, params=parameters)
response_content = response.text

print(response_content)

In [ ]:
# transform json to df
pd.json_normalize(response.json()['data'])

In [ ]:
# get all phone numbers to a list
phone_list = pd.json_normalize(response.json()['data'])['phone'].iloc[0]
phone_list

### 3) Search by Phone Number 📞
[Api Docs](https://docs.scrapeak.com/people-search-scraper/endpoints/searchbyphone) | Search by phone number

In [ ]:
phoneNumber = phone_list[0] # --> Format must be like this (123) 456-789
print('Phone:', phoneNumber)

#/ API endpoint
api_url = "https://app.scrapeak.com/v1/scrapers/people_search/search"

parameters = {"api_key": api_key, "search_by":"phone", "phone": phoneNumber}

#/ Make the API request
response = requests.get(api_url, params=parameters)
response_content = response.text

print(response_content)

In [ ]:
# transform json to df
pd.json_normalize(response.json()['data'])

# End Notebook